In [ ]:
import pandas as pd
import re

df = pd.read_csv('/secure/shared_data/rag_tnm_results/summary/5_folds_summary/brca_df.csv')

data_lst = []
for i, row in df.iterrows():
    pathology_report = row["text"]
    ground_truth = row["t"]
    filename = row["patient_filename"]

    question_text = f"""
    Based on the following pathology report for a breast cancer patient, determine the pathologic T stage (T1, T2, T3, or T4) for breast cancer, according to the AJCC Cancer Staging Manual (7th edition). 
    Choose from T1, T2, T3, T4.

    {pathology_report}
    """
    data = {"Question": question_text, "Answer": ground_truth, "Filename": filename}
    data_lst.append(data)

,Unnamed: 0,patient_filename,t,text,type,n
0,46,TCGA-3C-AALI.84E6A935-1A49-4BC1-9669-3DEA161CF6FC,1,Path No.: Date Obtained: (Age: ). Date Receive...,BRCA,1
1,47,TCGA-3C-AALJ.265E5A9A-64FD-4B86-89BC-5E89F253C118,1,Path No.: Date Obtained: (Age: ). Date Receive...,BRCA,1
2,48,TCGA-3C-AALK.F43B01E6-E1DB-44B1-8003-93870606346A,0,Path No.: Date Obtained: (Age: ). Date Receive...,BRCA,-1
3,127,TCGA-4H-AAAK.8894688F-7167-48A1-BB1B-FC219B7675C2,1,Procedure: Left radical mastectomy. Preoperati...,BRCA,2
4,245,TCGA-5L-AAT0.F9B6971F-23C0-465F-BFEC-778BF228A1AE,1,Gender: Female. Color: White. Origin: Nature o...,BRCA,0
...,...,...,...,...,...,...
1026,1512,TCGA-WT-AB41.FDB00212-9782-4ACE-AD59-019384297A2F,1,SURG PATH #: SPECIMEN CLASS: ALT ID #: SEX: F....,BRCA,-1
1027,1513,TCGA-WT-AB44.B7EB0E0B-46C5-43C0-A78D-FB094290765A,0,SURG PATH #: SPECIMEN CLASS: ALT ID #: SEX: F....,BRCA,-1
1028,1514,TCGA-XX-A89A.5D85E578-64B4-4238-922E-802B8ED87800,2,Research Gross Description. Research Dx. Left ...,BRCA,0
1029,1515,TCGA-Z7-A8R5.7726F7AA-88A8-4DD6-B322-6FC68893E0D2,2,ADDENDUM. Addendum #1. Entered. BREAST CANCER ...,BRCA,1


In [3]:
df = pd.read_csv('/home/yl3427/cylab/SOAP_MA/data/mergedBioNLP2023.csv',
                        usecols=['File ID', 'Subjective', 'Objective', 'Assessment', 'Summary', 'cleaned_expanded_Summary', 'terms'])

df = df.fillna('').apply(lambda x: x.str.lower())
df['combined_summary'] = df['Summary'] + df['cleaned_expanded_Summary'] + df['terms']

mi = ["myocardial infarction", "elevation mi", "non-stemi", " NSTEMI", " stemi"]
chf = ["congestive heart failure", " chf", "HFrEF", "HFpEF"]
pulmonary_embolism = ["pulmonary embolism"]
pulmonary_hypertension = ["pulmonary hypertension", "pulmonary htn"]
sepsis = ["sepsis", "septic shock"]
urosepsis = ["urosepsis"]
meningitis = ["meningitis"]
aki = ["acute kidney injury", " aki", "acute renal failure", " arf"] # -> Acute tubular necrosis (ATN)인가 아닌가
atn = ["acute tubular necrosis", " atn"]
pancreatitis = ["pancreatitis"]
gi_bleed = ["gastrointestinal bleed", "gi bleed"]
hepatitis = ["hepatitis", " hep"]
cholangitis = ["cholangitis"]
asp_pneumonia = ["aspiration pneumonia"]

prob_dict = {'sepsis': sepsis, 
                'acute kidney injury': aki, 
                'pancreatitis': pancreatitis, 
                'gastrointestinal bleed': gi_bleed,
                "congestive heart failure": chf}

ids = set()
for name, lst in prob_dict.items():
    problem_terms = lst
    problem_terms = [term.lower() for term in problem_terms]

    # Use the first term as the primary term to check in the combined summary.
    primary_term = problem_terms[0]

    # Build a regex pattern that matches any of the problem terms.
    # pattern = '|'.join(problem_terms)
    pattern = '|'.join(re.escape(term) for term in problem_terms)

    mask = (
        df['combined_summary'].str.contains(pattern, na=False) &
        ~df['Subjective'].str.contains(pattern, na=False) &
        ~df['Objective'].str.contains(pattern, na=False)
    )

    filtered_df = df[mask]

    ids.update(filtered_df['File ID'])

df = df[df['File ID'].isin(ids)]
df.to_csv('/home/yl3427/cylab/SOAP_MA/Input/SOAP_5_problems.csv', index=False)
print(df.shape)

(351, 8)


In [2]:
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import re
from typing import Optional, Union, List, get_origin, get_args, Any, Dict, Literal
import inspect
import asyncio
import json
import logging
import pandas as pd
from pydantic import BaseModel, Field, create_model
import math
# import demjson3


def generate_tools_spec(*functions):
    # Mapping of Python types to JSON Schema types
    type_map = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null"
    }
    tools = []
    
    for func in functions:
        func_name = func.__name__
        func_description = (func.__doc__ or "").strip()
        sig = inspect.signature(func)
        
        properties = {}
        required = []
        
        for param in sig.parameters.values():
            # Skip *args and **kwargs as they cannot be described in JSON schema easily
            if param.kind in (inspect.Parameter.VAR_POSITIONAL, inspect.Parameter.VAR_KEYWORD):
                continue
            
            param_name = param.name
            annotation = param.annotation
            json_type = "string"  # default type for fallback

            if annotation is not inspect._empty:
                origin = get_origin(annotation)
                
                # Handle Literal types (e.g., Literal["T", "N", "M"])
                if origin is Literal:
                    literal_args = get_args(annotation)
                    
                    # If all literal args are strings, produce a string enum
                    if all(isinstance(arg, str) for arg in literal_args):
                        properties[param_name] = {
                            "type": "string",
                            "enum": list(literal_args)
                        }
                    # If all are integers, produce an integer enum, etc.
                    elif all(isinstance(arg, int) for arg in literal_args):
                        properties[param_name] = {
                            "type": "integer",
                            "enum": list(literal_args)
                        }
                    else:
                        # Fallback if the Literal contains mixed or unsupported types
                        properties[param_name] = {"type": "string"}
                
                # Handle Optional[X] or Union[X, None]
                elif origin is Union:
                    union_args = [t for t in get_args(annotation) if t is not type(None)]
                    if len(union_args) == 1:
                        # e.g. Optional[str] -> just str
                        real_type = union_args[0]
                        origin2 = get_origin(real_type)
                        
                        if origin2 is Literal:
                            # If inside an Optional[Literal[...]]
                            literal_args = get_args(real_type)
                            if all(isinstance(arg, str) for arg in literal_args):
                                properties[param_name] = {
                                    "type": "string",
                                    "enum": list(literal_args)
                                }
                            elif all(isinstance(arg, int) for arg in literal_args):
                                properties[param_name] = {
                                    "type": "integer",
                                    "enum": list(literal_args)
                                }
                            else:
                                properties[param_name] = {"type": "string"}
                        else:
                            # Map direct type to JSON schema
                            json_type = type_map.get(real_type, "string")
                            properties[param_name] = {"type": json_type}
                    else:
                        # More complex Unions not automatically handled; fallback to string
                        properties[param_name] = {"type": "string"}
                
                # If it's a known type (str, int, etc.)
                elif annotation in type_map:
                    json_type = type_map[annotation]
                    properties[param_name] = {"type": json_type}
                
                # Handle typing.List[...] or typing.Dict[...] 
                elif origin in type_map:
                    json_type = type_map[origin]
                    if json_type == "array":
                        # For list[...] or array
                        item_type = "string"
                        args = get_args(annotation)
                        if args and args[0] in type_map:
                            item_type = type_map[args[0]]
                        properties[param_name] = {
                            "type": "array",
                            "items": {"type": item_type}
                        }
                    elif json_type == "object":
                        # For dict[...] or any unhandled complex mapping
                        properties[param_name] = {"type": "object"}
                
                else:
                    # Fallback if we can't detect the type
                    properties[param_name] = {"type": "string"}
            
            else:
                # No annotation; assume string
                properties[param_name] = {"type": "string"}

            # Mark as required if no default value
            if param.default is inspect._empty:
                required.append(param_name)
        
        tool_dict = {
            "type": "function",
            "function": {
                "name": func_name,
                "description": func_description,
                "parameters": {
                    "type": "object",
                    "properties": properties
                }
            }
        }
        if required:
            tool_dict["function"]["parameters"]["required"] = required
        
        tools.append(tool_dict)
    
    return tools


async def extract_information(info_to_extract: List[str], pathology_text: str) -> Dict[str, str]:
    """
    Extracts relevant information from a given pathology text.

    Args:
        info_to_extract (List[str]): A list of information fields to be extracted,
            e.g. ["tumor_size", "depth_of_invasion", ...].
        pathology_text (str): The full text of the pathology report.
    
    Returns:
        Dict[str, str]: A dictionary mapping each requested field to the extracted information.
    """
    client = AsyncOpenAI(base_url="http://localhost:8000/v1", api_key="dummy")
    model_name = "meta-llama/Llama-3.3-70B-Instruct"

    async def gather_responses():
        tasks = []
        for field in info_to_extract:
            prompt = f"""You are given a pathology report:

\"\"\"{pathology_text}\"\"\"

Please extract the information for the field: {field}.
Provide a concise answer containing only the relevant information for that field.
"""
            tasks.append(
                client.chat.completions.create(
                    messages=[{"role": "user", "content": prompt}],
                    model=model_name,
                    temperature=0.1,
                    max_tokens=500,
                )
            )

        # Run all extraction queries in parallel
        responses = await asyncio.gather(*tasks)

        # Build a dictionary of field -> extracted text
        extracted_info = {}
        for field, response in zip(info_to_extract, responses):
            # Depending on your LLM library's return format, adjust how you access content.
            raw_answer = response.choices[0].message.content.strip()
            extracted_info[field] = raw_answer

        return extracted_info

    # Run the async logic and return the results
    return await gather_responses()

def compare_numerical_values(
    value: float,
    min_value: float = None,
    max_value: float = None,
    inclusive_min: bool = True,
    inclusive_max: bool = True
) -> bool:
    """
    Compares a given numeric value against optional minimum and maximum thresholds.
    Args:
        value (float): The numeric value to compare.
        min_value (float, optional): The lower threshold. 
                                     If None, no lower bound check is performed.
        max_value (float, optional): The upper threshold. 
                                     If None, no upper bound check is performed.
        inclusive_min (bool): Whether the comparison with min_value 
                              should be inclusive (value >= min_value) 
                              or exclusive (value > min_value).
        inclusive_max (bool): Whether the comparison with max_value 
                              should be inclusive (value <= max_value) 
                              or exclusive (value < max_value).

    Returns:
        bool: True if the value satisfies all specified boundary conditions; 
              False otherwise.

    Examples:
        compare_numerical_values(3.2, min_value=2, max_value=5) 
            -> True (assuming inclusive checks)
        compare_numerical_values(2, min_value=2, max_value=5, inclusive_min=False)
            -> False, since 2 is not strictly > 2
    """
    if min_value is not None:
        if inclusive_min:
            if value < min_value:
                return False
        else:
            if value <= min_value:
                return False

    if max_value is not None:
        if inclusive_max:
            if value > max_value:
                return False
        else:
            if value >= max_value:
                return False

    return True

def produce_final_staging_response(agent_response: str) -> Dict[str, str]:
    """
    Takes the agent's final response and reformat it into a JSON schema with 'reasoning' and 'stage' as keys.

    Args:
        agent_response (str): The final response from the agent (after all internal processing).

    Returns:
        Dict[str, str]: A dictionary containing two keys: 'reasoning' and 'stage'.
    """

    # Example: we create a second LLM client specifically for formatting the final output
    formatting_client = OpenAI(base_url="http://localhost:8000/v1", api_key="dummy")
    formatting_model = "meta-llama/Llama-3.3-70B-Instruct"

    prompt = f"""You are given the final reasoning and conclusions about a cancer T-staging task:

\"\"\"{agent_response}\"\"\"

Please provide valid JSON (and ONLY JSON, without extra text) with the following structure:
{{
"reasoning": "A brief explanation of the reasoning that led to the stage conclusion.",
"stage": "The final T stage (e.g. T1, T2, T3...)"
}}

Make sure the output is strictly valid JSON.
"""
        
    class ResponseStage(BaseModel):
        reasoning: str = Field(
            description="A step-by-step explanation for how you arrived at the predicted T stage."
        )
        stage: Literal["T1", "T2", "T3", "T4"] = Field(
            description="The final predicted T stage (T1, T2, T3, or T4)."
        )


    response = formatting_client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=formatting_model,
        temperature=0.1,
        extra_body={"guided_json": ResponseStage.model_json_schema()}
        )

    return response.choices[0].message.content


available_tools = {
    "extract_information": extract_information,

}
tools = generate_tools_spec(*available_tools.values())

In [3]:
tools

[{'type': 'function',
  'function': {'name': 'extract_information',
   'description': 'Extracts relevant information from a given pathology text.\n\n    Args:\n        info_to_extract (List[str]): A list of information fields to be extracted,\n            e.g. ["tumor_size", "depth_of_invasion", ...].\n        pathology_text (str): The full text of the pathology report.\n    \n    Returns:\n        Dict[str, str]: A dictionary mapping each requested field to the extracted information.',
   'parameters': {'type': 'object',
    'properties': {'info_to_extract': {'type': 'array',
      'items': {'type': 'string'}},
     'pathology_text': {'type': 'string'}},
    'required': ['info_to_extract', 'pathology_text']}}}]

# MedicalQA 데이터, 5지선다만 골라내기

In [5]:
import pandas as pd
import re

# Read each CSV file into a DataFrame
df1 = pd.read_csv('/home/yl3427/cylab/SOAP_MA/Input/step1_ALL.csv')
df2 = pd.read_csv('/home/yl3427/cylab/SOAP_MA/Input/step2_ALL.csv')
df3 = pd.read_csv('/home/yl3427/cylab/SOAP_MA/Input/step3_ALL.csv')

# Reassign the qn_num values with a prefix for each file
df1['qn_num'] = '1_' + df1['qn_num'].astype(str)
df2['qn_num'] = '2_' + df2['qn_num'].astype(str)
df3['qn_num'] = '3_' + df3['qn_num'].astype(str)

# Concatenate the DataFrames into one
merged_df = pd.concat([df1, df2, df3], ignore_index=True)

# Optionally, view the merged DataFrame
print(len(merged_df))

# Filter rows based on the rule
filtered_rows = []

for _, row in merged_df.iterrows():
    matches = re.findall(r'\b[A-Z](?=[.:])', row['choice'])
    if set(matches) == {'A', 'B', 'C', 'D', 'E'}:
        filtered_rows.append(row)

# Convert the list of rows to a new DataFrame
filtered_df = pd.DataFrame(filtered_rows)

# Optionally, view the new DataFrame
print(len(filtered_df))
filtered_df.to_csv('/home/yl3427/cylab/SOAP_MA/Input/filtered_merged_QA.csv', index=False)

376
284


In [6]:
len(filtered_df.qn_num.unique())

284